In [1]:
from tgate_counter import count_tgate
from prepare_circuit import uniform_layered_m_body, dist_select_m_body
from statistics import mean, pstdev
from multiprocessing import cpu_count
# 標準のmultiprocessingの代わりにmpを使わないとトップスコープの名前が解決されない
import multiprocess as mp
import pyzx as zx

def print_result(result_dict):
    size = result_dict["size"]
    count = result_dict["count"]
    before = result_dict["before"]
    mean = result_dict["mean"]
    stdev = result_dict["stdev"]
    parallel = result_dict["parallel"]
    print("=====")
    print("stat: size ->", 2**size, ", count ->", count, ", parallel ->", parallel)
    print("before t-depth:", before)
    print("after t-depth:", mean, "+/-", stdev)
    imp = (1 - mean/before)*100
    print("improvement: ", imp, "%")

def reduce_qasm(qasm_str: str) -> int:
    qasm_circuit = zx.Circuit.from_qasm(qasm_str)
    g = qasm_circuit.to_graph()
    zx.draw(g) # draw circuit
    zx.full_reduce(g)
    g.normalize()
    c_opt = zx.extract_circuit(g.copy())
    after_qasm = c_opt.to_qasm()
    return count_tgate(after_qasm)[1]

def experiment_m_body(size: int, interaction: int, parallel: int, count: int):
    # prepare qasms before optimization
    before_qasms: list[str] = []
    for _ in range(count):
        if parallel == 0:
            qasm = uniform_layered_m_body(size, interaction)
        else:
            qasm = dist_select_m_body(size, interaction, parallel)
        before_qasms.append(qasm)
    before = mean([count_tgate(qasm)[1] for qasm in before_qasms])
    # concurrent optimization
    with mp.Pool(cpu_count()) as pool:
        afters: list[int] = pool.map(reduce_qasm, before_qasms)
    avg = mean(afters)
    stdev = pstdev(afters)
    return {
        "size": size,
        "count": count,
        "before": before,
        "mean": avg,
        "stdev": stdev,
        "parallel": parallel
    }

prepare-circuit version 1.0.12


In [2]:
print_result(experiment_m_body(4, 1, 0, 10))
print_result(experiment_m_body(4, 1, 1, 10))
print_result(experiment_m_body(4, 1, 2, 10))
print_result(experiment_m_body(4, 1, 3, 10))

=====
stat: size -> 16 , count -> 10 , parallel -> 0
before t-depth: 92
after t-depth: 98.1 +/- 3.3896902513356584
improvement:  -6.630434782608696 %
=====
stat: size -> 16 , count -> 10 , parallel -> 1
before t-depth: 92
after t-depth: 86.9 +/- 9.730878685915265
improvement:  5.543478260869561 %
